In [1]:
import plotly.express as px
# import cv2, sys
import sys, os
import numpy as np
from numpy import sin,cos,deg2rad
from IPython.display import display, Image
import ipywidgets as widgets


import plotly.graph_objects as go

In [3]:
# Importar path anterior
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# Import from created files

from CinematicaInversa.Servo import Servo
from CinematicaInversa.Plate import Plate
from CinematicaInversa.BallAndPlate import BallAndPlate
from CinematicaInversa.MatrixFunctions import rotate3d, translate3d





# from PlotFunctions import plot3dMatrix, text3d, drawLinePoints, drawCircle, plotRadius
# from MatrixFunctions import rotate3d, translate3d



In [4]:
Servo1 = Servo(servoIndex=22,zeroAngle=1,servoPos=np.matrix([[0,10,0]]).T,remote=True)
Servo2 = Servo(servoIndex=1,zeroAngle=0,servoPos=np.matrix([[10*sin(deg2rad(120)),10*cos(deg2rad(120)),0]]).T,remote=True)
Servo3 = Servo(servoIndex=2,zeroAngle=0,servoPos=np.matrix([[10*sin(deg2rad(240)),10*cos(deg2rad(240)),0]]).T,remote=True)
plate = Plate(height=10)

ballAndPlate = BallAndPlate(Servo1,Servo2,Servo3,plate,remote=True,remoteType=None)

ballAndPlate.setHeight(10)
ballAndPlate.setAngle(0,0)

# Ex pos J1
ballAndPlate.Servo1.pos

# Ex pos J3
ballAndPlate.Plate.magnet1Pos

matrix([[ 0.],
        [10.],
        [10.]])

In [5]:
def getPointsofMatrix(matrix):
    return matrix[0].item(),matrix[1].item(),matrix[2].item()



def drawCircle(fig,cx,cy,cz,radius,angleX,angleY,angleZ,color='black',**kwargs):
    theta = np.linspace(0, 2*np.pi, 100)


    # compute x1 and x2
    x = radius*np.cos(theta)
    y = radius*np.sin(theta)
    z = 0*theta

    xn = []
    yn = []
    zn = []
    for i in range(len(x)):
        coord = np.matrix([[x[i],y[i],z[i]]]).T

        res = rotate3d(coord,angleX,angleY,angleZ)
        xi,yi,zi = getPointsofMatrix(res.A1)

        xn.append(xi+cx)
        yn.append(yi+cy)
        zn.append(zi+cz)
        
    fig.add_trace(
        go.Scatter3d(x=xn,
                     y=yn,
                     z=zn,
                     mode='lines',    
                     line=dict(width=10,color=color)
                     ,**kwargs)
    )

In [6]:
ballAndPlate.J2_1['BestPos']

matrix([[0.        ],
        [3.98352961],
        [3.578     ]])

In [12]:
# Figura vazia 
fig = go.Figure()


# Desenha as juntas J1
j1_x1,j1_y1,j1_z1 = getPointsofMatrix(ballAndPlate.Servo1.pos)
j1_x2,j1_y2,j1_z2 = getPointsofMatrix(ballAndPlate.Servo2.pos)
j1_x3,j1_y3,j1_z3 = getPointsofMatrix(ballAndPlate.Servo3.pos)


fig.add_trace(
    go.Scatter3d(x=[j1_x1,j1_x2,j1_x3],
                 y=[j1_y1,j1_y2,j1_y3],
                 z=[j1_z1,j1_z2,j1_z3],
                 name='Junta J1',
                 mode='markers',
                 marker=dict(color='blue'))
)






# Desenha as juntas J3
j3_x1,j3_y1,j3_z1 = getPointsofMatrix(ballAndPlate.Plate.magnet1Pos)
j3_x2,j3_y2,j3_z2 = getPointsofMatrix(ballAndPlate.Plate.magnet2Pos)
j3_x3,j3_y3,j3_z3 = getPointsofMatrix(ballAndPlate.Plate.magnet3Pos)

fig.add_trace(
    go.Scatter3d(x=[j3_x1,j3_x2,j3_x3],
                 y=[j3_y1,j3_y2,j3_y3],
                 z=[j3_z1,j3_z2,j3_z3],
                 name='Junta J3',
                 mode='markers',
                 marker=dict(color='#cca000'))
)

# Desenha as juntas J2
j2_x1,j2_y1,j2_z1 = getPointsofMatrix(ballAndPlate.J2_1['BestPos'])
j2_x2,j2_y2,j2_z2 = getPointsofMatrix(ballAndPlate.J2_2['BestPos'])
j2_x3,j2_y3,j2_z3 = getPointsofMatrix(ballAndPlate.J2_3['BestPos'])

fig.add_trace(
    go.Scatter3d(x=[j2_x1,j2_x2,j2_x3],
                 y=[j2_y1,j2_y2,j2_y3],
                 z=[j2_z1,j2_z2,j2_z3],
                 name='Junta J2',
                 mode='markers',
                 marker=dict(color='green'))
)

#Desenha os links
fig.add_trace(
    go.Scatter3d(x=[j1_x1,j2_x1,j3_x1],
                 y=[j1_y1,j2_y1,j3_y1],
                 z=[j1_z1,j2_z1,j3_z1],
                 mode='lines',    
                 name='links1',
                 line=dict(width=8,color='purple')))

fig.add_trace(
    go.Scatter3d(x=[j1_x2,j2_x2,j3_x2],
                 y=[j1_y2,j2_y2,j3_y2],
                 z=[j1_z2,j2_z2,j3_z2],
                 mode='lines',    
                 name='links2',
                 line=dict(width=8,color='purple')))

fig.add_trace(
    go.Scatter3d(x=[j1_x3,j2_x3,j3_x3],
                 y=[j1_y3,j2_y3,j3_y3],
                 z=[j1_z3,j2_z3,j3_z3],
                 mode='lines',    
                 name='links3',
                 line=dict(width=8,color='purple')))

# Desenha circulo superficie
xcenter,ycenter,zcenter = getPointsofMatrix(ballAndPlate.Plate.centerPos)
drawCircle(fig,
           cx=xcenter,
           cy=ycenter,
           cz=zcenter,
           radius=ballAndPlate.Plate.radius,
           angleX=deg2rad(ballAndPlate.Plate.xAngle),
           angleY=deg2rad(ballAndPlate.Plate.yAngle),
           angleZ=0,
           color='black',
           name='Superficie'
          )

# Desenha circulo Base
xcenter,ycenter,zcenter = [0,0,0]
drawCircle(fig,
           cx=xcenter,
           cy=ycenter,
           cz=zcenter,
           radius=ballAndPlate.Plate.radius,
           angleX=0,
           angleY=0,
           angleZ=0,
           color='black',
           name='Base')


fig.update_layout(autosize=False,margin=dict(t=100, r=200, l=100, b=20), width=900  ,           
                  scene=dict(xaxis= dict(range=[-20,20],showgrid=True,visible=True),
                             yaxis= dict(range=[-20,20],showgrid=True,visible=True),
                             zaxis = dict(range=[-1,15],showgrid=True,visible=True),
                             aspectmode='data'
                             
                              ))


camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=-0.2),
    eye=dict(x=1, y=1, z=0.6)
)

fig.update_layout(scene_camera=camera)
fig.show()

In [110]:
fig.layout.scene.camera

layout.scene.Camera()